In [ ]:
import re
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import snscrape.modules.twitter as sntwitter
import pandas as pd
import time
import datetime

# Creating list to append tweet data 
news_report = []
count = 1

# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('from:Cointelegraph').get_items()):
    if tweet.date < datetime.datetime(2022, 3, 1, 0, 0, 0, tzinfo=datetime.timezone.utc):  # Scraping end date
        break

    if datetime.datetime(2022, 4, 1, 0, 0, 0, tzinfo=datetime.timezone.utc) > tweet.date:  # Scraping start date
        try:
            r = requests.get(re.search("(?P<url>https?://[^\s]+)", tweet.content.replace('\n', ' ')).group("url"))
            if 'https://cointelegraph.com/news/' in r.url:
                # driver = webdriver.Edge(executable_path='C:/Users/USER/Downloads/edgedriver_win64/msedgedriver.exe')
                driver = webdriver.Chrome(executable_path='C:/Users/USER/Downloads/chromedriver_win32/chromedriver.exe')
                driver.get(r.url)

                html = driver.page_source
                soup = BeautifulSoup(html)
                content = ''

                title = soup.find('h1', class_='post__title').get_text()
                abstract = soup.find('p', class_='post__lead').get_text().strip()
                post = soup.find('div', class_='post-content')
                for content_ in post.find_all('p'):
                    content += content_.get_text('') + ' '

                driver.close()

            if i%30==0:
                news_df = pd.DataFrame(news_report, columns=['Datetime', 'Tweet Id', 'Text', 'Username', 'Article Title', 'Article Abstract', 'Article Content'])
                news_df.to_csv("./cointelegraph_article(100" + str(count) +").csv", sep=",")
                count += 1
                time.sleep(80) #100

            if i%8==0:
                time.sleep(60) #80

            if i%3==0:
                time.sleep(50)

            news_report.append([tweet.date, tweet.id, tweet.content.replace('\n', ' '), tweet.user.username, title, abstract, content])
        except:  # possible with NameError or no URL in tweet
            print(tweet.date)
            title = ''
            abstract = ''
            content = ''
            news_report.append([tweet.date, tweet.id, tweet.content.replace('\n', ' '), tweet.user.username, title, abstract, content])

try:
    driver.quit()
except:
    pass

news_df = pd.DataFrame(news_report, columns=['Datetime', 'Tweet Id', 'Text', 'Username', 'Article Title', 'Article Abstract', 'Article Content'])
news_df.to_csv("./cointelegraph_article(100" + str(count) + ")_final.csv", sep=",")